In [43]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sb

In [44]:
mnist= tf.keras.datasets.mnist
(train_images,train_labels) , (test_images,test_labels)=mnist.load_data()

##### Transform the train images and test images. (Place 28 * 28 randomly in 56 * 56 grid)


In [45]:
temp=[]
for i in range(len(train_images)):
    (x,y) = (random.randint(0,28),random.randint(0,28))
    temp_img= np.array([[ 0 for i in range(56)] for j in range(56)]) 
    temp_img[x:x+28 , y:y+28] += train_images[i]
    temp.append(temp_img)
    
train_images=np.array(temp)
print('train_images.shape : ',train_images.shape)
train_images = train_images.reshape((60000,56,56,1))
train_images = train_images/255

train_images.shape :  (60000, 56, 56)


In [46]:
temp=[]
for i in range(len(test_images)):
    (x,y) = (random.randint(0,28),random.randint(0,28))
    temp_img= np.array([[ 0 for i in range(56)] for j in range(56)]) 
    temp_img[x:x+28 , y:y+28] += test_images[i]
    temp.append(temp_img)
    
test_images=np.array(temp)
print('test_images.shape : ',test_images.shape)
test_images = test_images.reshape((10000,56,56,1))
test_images = test_images/255

test_images.shape :  (10000, 56, 56)


In [ ]:
model= tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64,3,input_shape=(56,56,1), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(128,3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32,activation= 'relu'))
model.add(tf.keras.layers.Dense(10))


model.summary()

In [72]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = tf.keras.optimizers.Adam(learning_rate=0.001)
metrics = ['accuracy']
epochs=5
batchSize=64

model.compile(loss=loss,optimizer=optim,metrics=metrics)
model.fit(train_images,train_labels,batch_size=batchSize,epochs=epochs,shuffle=True)



Epoch 1/5
938/938 [==============================] - 36s 38ms/step - loss: 0.6435 - accuracy: 0.7859
Epoch 2/5
938/938 [==============================] - 36s 38ms/step - loss: 0.1629 - accuracy: 0.9513
Epoch 3/5
938/938 [==============================] - 36s 38ms/step - loss: 0.0982 - accuracy: 0.9693
Epoch 4/5
938/938 [==============================] - 36s 38ms/step - loss: 0.0638 - accuracy: 0.9797
Epoch 5/5
938/938 [==============================] - 36s 38ms/step - loss: 0.0444 - accuracy: 0.9855


In [74]:
model.evaluate(test_images,test_labels,batchSize)

157/157 [==============================] - 2s 15ms/step - loss: 0.1408 - accuracy: 0.9594


[0.14080668985843658, 0.9593999981880188]